# Homogeneisation - main

L'objectif est de mettre à disposition des chercheurs une interface permettant :
- le calcul de comportements homogénéisés sur des microstructures données à l'aide de différents modèles pré-implémentés, manuellement ou à partir d'un fichier texte,
- de faciliter des études parmaétriques sur ces mêmes modèles et microstructures,
- de déterminer des paramètres idéaux pour atteindre des comportements homogénéisés cibles(TODO),
- de fournir une description des modèles utilisés, de leur forces et de leurs limites.

Ce code contient plusieurs sections. **Lorsque le code est lancé pour la première fois sur un nouveau kernel, les sections suivantes doivent être executées dans l'ordre** :
- Importation des classes et des modules utiles
- Fonctions utiles 

Les sections suivantes sont indépendantes. Si une section est lancée pour la première fois, ses cellules doivent être executées dans l'ordre.

---

## I- Importation des classes et des modules utiles

In [ ]:
!pip install ipywidgets # Installation du package permettant la gestion des widgets
from IPython.display import clear_output, display
clear_output()
print("Package downloaded")

In [ ]:
import ipywidgets as widgets
from classes import *
from os import listdir
import pandas as pd

In [ ]:
print("Modèles importés : {}".format(list_models))

---

## II- Fonctions utiles

In [ ]:
parameters_name = {'K': 'Bulk modulus K', 'G': 'Shear modulus G', 'E': 'Young modulus E', 'nu': "Poisson's ratio nu"}
parameters_name = {'K': 'Bulk modulus K', 'G': 'Shear modulus G', 'E': 'Young modulus E', 'nu': "Poisson's ratio " + r'\(\nu\)'}
parameters_name_bis = {value: key for (key, value) in parameters_name.items()}

def gen_tab_behavior():
    """
    Routine qui génère un widget de type 'tab' permettant le choix des paramètres associés aux comportements. 
    Chaque onglet généré correspond à un comportement du dictionnaire 'dict_behaviors' implémenté dans 'classes.py'.
    La fonction renvoie :
    - une liste de liste de widgets (list_widgets) contenant autant de listes que d'onglet. Chaque liste contient les widgets générant les paramètres asssociés au comportement 
    de l'onglet correspondant.
    - un widget tab correspondant au résultat final à afficher.    
    """
    behaviors_str = list(dict_behaviors.keys()) # Liste des noms des comportements déjà implémentés, définis dans classes.py
    list_widgets = [] # Liste de liste, chaque liste correspond à un onglet du tab et contient les widgets de cet onglet non formatés
    tab_titles = [] # Nom des onglets, chaque onglet est associé à un type de comportement
    # Construction des widgets associés à chaque onglet
    for behavior_str in behaviors_str:
        widgets_onglet = []
        parameters = dict_behaviors[behavior_str] # Paramètres associés au comportement (exemple : ['K', 'G'] pour Isotropic)
        for parameter in parameters:
            w = widgets.FloatSlider(value=1, min=0.01, max=1000, step=0.01) # Widget associé au paramètre parameter
            if parameter == 'nu':
                w.max = 0.5
                w.value = 0.3
            w_label = widgets.Label(value=parameters_name[parameter])
            widgets_onglet.append(widgets.HBox([w_label, w]))
        list_widgets.append(widgets_onglet)
        tab_titles.append(behavior_str)
    # Création du tab
    tab = widgets.Tab()
    tab.children = [widgets.HBox(w) for w in list_widgets]
    for pos, title in enumerate(tab_titles):
        tab.set_title(pos, title)
    return list_widgets, tab

def read_behavior(tab, list_widgets):
    """
    Fonction qui, à partir d'un widget 'tab' comme celui construit par la fonction précédente, renvoie le dictionnaire 'behavior' construit par l'utilisateur.
    """
    behavior_int = tab.selected_index # Onglet ouvert par l'utilisateur
    widgets_parameters = list_widgets[behavior_int] # Widgets des paramètres de l'onglet ouvert
    behavior = {parameters_name_bis[w.children[0].value] : w.children[1].value for w in widgets_parameters}
    return behavior

def gen_tab_type():
    """
    Routine qui génère un widget de type 'tab' permettant le choix des paramètres associés aux types d'inclusions (aspect ratio, orientation, etc..). 
    TODO : Inclure l'orientation
    Chaque onglet généré correspond à un type du dictionnaire 'dict_types' implémenté dans 'classes.py'.
    La fonction renvoie :
    - une liste de liste de widgets (list_widgets) contenant autant de listes que d'onglets. Chaque liste contient les widgets générant les paramètres asssociés au comportement 
    de l'onglet correspondant.
    - un widget tab correspondant au résultat final à afficher.    
    """
    list_widgets = [] # Liste de listes, chaque liste correspond à un onglet et contient les widgets de cet onglet
    # Construction des widgets associés à chaque onglet
    for type_int in dict_types.keys():
        if type_int == 0:
            # Sphères, pas besoin de paramètres supllémentaires
            list_widgets.append([])
        elif type_int == 1:
            # Oblate
            w_label = widgets.Label(value="Aspect ratio")
            w_aspect_ratio = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.5)
            list_widgets.append([w_label, w_aspect_ratio])
        elif type_int == 2:
            # Prolate
            w_label = widgets.Label(value="Aspect ratio")
            w_aspect_ratio = widgets.FloatSlider(min=1.01, max=10, step=0.01, value=1.5)
            list_widgets.append([w_label, w_aspect_ratio])
    # Création du tab
    tab = widgets.Tab()
    # Attribution des onglets
    tab.children = [widgets.HBox(w) for w in list_widgets]
    # Attribution des noms des onglets
    for pos, title in dict_types.items():
        tab.set_title(pos, title)
    return list_widgets, tab         

def read_type(tab, list_widgets):
    """
    Fonction qui, à partir d'un widget 'tab' comme celui construit par la fonction gen_tab_type, renvoie le type d'inclusion et le rapport d'apsect.
    """
    type_int = tab.selected_index # Onglet ouvert par l'utilisateur
    widgets_parameters = list_widgets[type_int]
    try:
        aspect_ratio = widgets_parameters[1].value
    except:
        # Il n'y a pas de rapport de forme, cas des sphères par exemple
        aspect_ratio = 1
    return type_int, aspect_ratio

def str_to_model(model_name):
    """
    Fonction qui renvoie l'instance de classe Model associée au nom de classe model_name (str).
    """
    for Model in list_models:
        model = Model()
        if model.name.upper() == model_name.upper():
            return model

---

## III- Calcul de comportement homogénéisé de microstructures
Cette section permet la génération manuelle de microstructure et le calcul de comportement homogénéisé avec les modèles disponibles.

In [ ]:
dict_inclusions = {}
# Initialise la liste des inclusions créees. dict_inclusions est sous la forme {nam_inclusion (str): inclusion (Inclusion)}

### Construction d'une inclusion

In [ ]:
# Nom de l'inclusion
w_label = widgets.Label(value='Inclusion name')
n_inclusion = 0 # Identifiant servant à donner automatiquement un nom unique à chaque inclusion
w_name = widgets.Text(value='inclusion '+str(n_inclusion))
display(w_label, w_name)

# Type d'inclusion
display(widgets.Label(value='Inclusion type'))
widgets_type, tab_type = gen_tab_type() # Génération du widgets tab de choix du type d'inclusion
display(tab_type)

# Comportement de l'inclusion
caption = widgets.Label(value='Inclusion behavior')
display(caption)
list_widgets, tab = gen_tab_behavior()
display(tab)

# Génération de l'inclusion
button_generate_inclusion = widgets.Button(description="Generate Inclusion")
output = widgets.Output()
display(button_generate_inclusion, output)
def generate_inclusion(b):
    """
    Fonction appelée lors d'un click sur le bouton, génère une inclusion avec les paramètres choisis.
    TODO : modifier le nom de l'inclusion automatiquement
    """
    global n_inclusion
    # Récupération des paramètres choisis
    output.clear_output()
    inclusion_name = w_name.value
    if inclusion_name in list(dict_inclusions.keys()):
        with output:
            print("Name already exists")
    else :
        type_inclusion, inclusion_aspect_ratio = read_type(tab_type, widgets_type)
        behavior = read_behavior(tab, list_widgets)
        inclusion = Inclusion(type_inclusion, behavior, name=inclusion_name, aspect_ratio=inclusion_aspect_ratio)
        dict_inclusions[inclusion_name] = inclusion
        with output:
            print("Inclusion generated: ", inclusion)
        # Mise à jour automatique du nom de l'inclusion
        n_inclusion += 1
        w_name.value = 'inclusion '+str(n_inclusion)
    
button_generate_inclusion.on_click(generate_inclusion)

### Construction d'une microstructure

In [ ]:
# Création de la variable contenant la microstructure
microstructure = None # Initialisation

# Fonctions liées aux appuis sur les boutons
def add_inclusion_to_structure(b):
    """
    Fonction appelée lors d'un click sur le bouton "Add inclusion", génère un widget associé à la fraction volumique de l'inclusion et l'ajoute au dictionnaire "widgets_f".
    Crée aussi un bouton permettant la suppression de l'inclusion et l'ajoute au dictionnaire buttons.
    Enfin, affiche la ligne de widgets correspondante.
    """
    out2.clear_output()
    inclusion = w_inclusions.value
    if inclusion in list(widgets_f.keys()):
        with out2:
            print("Already added")
    else:
        w_name = widgets.Label(inclusion.name)
        w_f = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f')
        w_b = widgets.Button(description="Remove inclusion")
        w_b.on_click(remove_inclusion)
        widgets_f[inclusion] = (w_name, w_f)
        buttons_suppress[w_b] = inclusion
        with out1:
            display(w_name, widgets.HBox([w_f, w_b]))

def add_inclusion_to_list(b):
    """
    Fonction appelée lors d'un clic sur le bouton 'generate inclusion' de la section précédente.
    met à jour le widget permettant le choix des inclusions à ajouter.
    """
    w_inclusions.options = list(dict_inclusions.values())
    
def remove_inclusion(b):
    """
    Fonction appelée lorsqu'un bouton 'Remove inclusion' est appelé. 
    Repère l'inclusion associée au bouton, ferme les widgets associés et supprime l'entrée du dictionnaire widgets_f
    """
    out2.clear_output()
    inclusion = buttons_suppress[b]
    w_name, w_f = widgets_f[inclusion]
    # Fermeture des widgets
    b.close()
    w_name.close()
    w_f.close()
    del widgets_f[inclusion]
    del buttons_suppress[b]

def generate_microstructure(b):
    """
    Génère la microstructure avec les paramètres choisis par l'utilisateur. 
    Si les fractions volumiques choisies ne sont pas cohérentes, affiche un message.
    Affiche une description de la microsructure créee.
    """
    global microstructure
    matrix_behavior = read_behavior(tab_m, widgets_m) # Lecture des widgets définis dans la section 'Comportement de la matrice'
    dict_inclusions = {}
    # Lecture des fractions volumiques choisies
    for inclusion, widgets in widgets_f.items():
        w_name, w_f = widgets
        f = w_f.value
        dict_inclusions[inclusion] = f
    # Génération de la microstructure
    out3.clear_output()
    try:
        microstructure = Microstructure(matrix_behavior, dict_inclusions)
        with out3:
            print("Microstructure generated\n" + str(microstructure))
            # Dessin de la microstructure TODO : ajouter le dessin d'inclusion ellipsoïdales
            microstructure.draw()
    except NameError:
        microstructure = None
        with out3:
            print("Inconsistent choice of volumic fractions")

In [ ]:
# Comportement de la matrice
caption = widgets.Label(value='Matrix behavior')
display(caption)
widgets_m, tab_m = gen_tab_behavior()
display(tab_m)

# Ajout d'inclusions
w_inclusions = widgets.Dropdown(options=list(dict_inclusions.values()), layout={'width': 'max-content'})
button_add_inclusion = widgets.Button(description="Add inclusion")
out1 = widgets.Output()
out2 = widgets.Output()
display(widgets.HBox([w_inclusions, button_add_inclusion, out2]), out1)
widgets_f = {} # Dictionnaire des inclusions ajoutées et de leurs widgets ('name','fraction volumique') associés
buttons_suppress = {} # Dictionnaire des boutons permettant de supprimer une inclusion de la structure et inclusions associés

button_add_inclusion.on_click(add_inclusion_to_structure)
button_generate_inclusion.on_click(add_inclusion_to_list)

# Génération de la microstructure
b_generate_structure = widgets.Button(description='Generate microstructure', layout={'width': 'max-content'})
# TODO : widget 'valid' qui indique en temps réel si les fractions volumiques choisies sont cohérentes
out3 = widgets.Output()
display(widgets.HBox([b_generate_structure]),out3)
b_generate_structure.on_click(generate_microstructure)


### Calcul du comportement homogénéisé

In [ ]:
# Choix du modèle
def test_models(b=None):
    """
    Fonction appelée lors d'un appui sur le bouton 'Generate microstructure' juste en haut.
    Teste les modèles disponibles sur la microstructure générée et met à jour la liste des modèles valides 'valid_models'.
    """
    valid_models = []
    if microstructure == None:
        # Vérifie que la microstructure a bien été créee
        return None
    for Model in list_models:
        model = Model()
        valid = model.check_hypothesis(microstructure)
        if valid:
            # La microstructure vérifie les hypothèses du modèle.
            valid_models.append((model.name, model))
    # Mise à jour du widget de séléection du modèle
    select_model.options = valid_models

valid_models = [] # Liste des modèles pouvant s'appliquer à la microstructure donnée, format : [(model_name, Model)]
select_model = widgets.Dropdown()
test_models()
label = widgets.Label(value="Select a model. Only compatible models will be displayed.")
display(label)
b_compute = widgets.Button(description='Compute behavior')
display(widgets.HBox([select_model, b_compute]))
output_behavior = widgets.Output()
display(output_behavior)

def compute_model(b):
    """
    Fonction appelée lors d'un appui sur le bouton 'Compute behavior'.
    Récupère le modèle choisi par l'utilisateur, calcule le comportement homogénéisé de la structure et l'affiche.
    """
    model = select_model.value
    homogenised_behavior = model.compute_h_behavior(microstructure)
    output_behavior.clear_output()
    with output_behavior:
        print("Homogenised behavior - {} model".format(model.name))
        print(homogenised_behavior)
        print("Hashin bounds")
        print(microstructure.Hashin_bounds())
    

b_generate_structure.on_click(test_models)
b_compute.on_click(compute_model)

### Comparaison de modèles

In [ ]:
# Fonction de tracé
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation) #Désactive le message de warning de matplotlib 

def draw_all_data(subplots):
    """
    Fonction qui trace le graphe des données dans la base de données subplots, dans le widgets de type output w_out.
    Le format de la bdd est explicité dans la section suivante.
    """
    global fig
    out_graph.clear_output()
    # Calcul du nombre de subplots
    parameters = list(subplots.keys())
    n_parameters = len(parameters)
    n_lines = n_parameters//2
    if n_parameters%2 == 0:
        None
    else:
        n_lines += 1
    with out_graph:
        fig, axs = plt.subplots(n_lines, 2, figsize=(20,n_lines*7))
        for index, parameter in enumerate(parameters):
            plt.subplot(n_lines, 2, index+1)
            list_data = subplots[parameter]
            for data in list_data:
                x = data[0]
                if len(x)>50:
                    # Représentation continue
                    plt.plot(data[0], data[1], label=data[2])
                else:
                    # Représentation discrète
                    plt.plot(data[0], data[1], '.', label=data[2])
            plt.xlabel("Volumic fraction")
            plt.ylabel(parameter)
            plt.grid(b=True)
            plt.title("Model comparison - "+parameter)
            plt.legend()
        plt.show()


In [ ]:
out_graph = widgets.Output()
subplot_data = {} # Format {parameter: subplot_data}, avec subplot_data = [[f_list, value, label]] et autant de liste que de modèles
plotted = [] # Liste des modèles et fichiers déjà tracés
fig = None

# Choix de la fraction volumique à faire varier
display(widgets.Label(value="Select an inclusion, then click on 'Start comparing'. The results of the compared models will be plotted against the volumic fraction of this inclusion."))
w_inclusion = widgets.Dropdown(layout={'width': 'max-content'})
w_setgraph = widgets.Button(description='Start comparing / Reset graph', layout={'width': 'max-content'})
display(widgets.HBox([w_inclusion, w_setgraph]))

# Mise à jour de la liste des inclusions lors de la génération d'une microstructure
def update_inclusions_list(b):
    """
    Routine appelée lors de l'appui sur le bouton 'Generate microstructure'.
    Met à jour la liste des inclusions de la microstructure (widget w_inclusion).
    """
    global subplot_data, plotted
    inclusions = microstructure.dict_inclusions
    w_inclusion.options = list(inclusions.keys())
    out_graph.clear_output()
    subplot_data = {}
    plotted = []

if microstructure!= None:
    # Condition permettant d'éviter l'erreur qui a lieu lorsque la section est compilée avant qu'une microstructure ne soit générée
    update_inclusions_list(None)
b_generate_structure.on_click(update_inclusions_list)

# Calcul de la liste des fractions volumiques compatibles
f_list = [] # Liste des fractions volumiques compatibles
inclusion = None # Inclusion choisie

def compute_f_list(b):
    """
    Routine appelée lors de l'appui sur le bouton 'Start comparing'.
    Récupère l'inclusion choisie, et calcule l'intervalle de fractions volumiques f_list compatible avec le reste des inclusions.
    """
    global f_list, inclusion, subplot_data, plotted
    inclusion = w_inclusion.value
    # Calcul de f_max
    f_max = 0.99
    dict_inclusions = microstructure.dict_inclusions
    for other_inclusion, f in list(dict_inclusions.items()):
        if other_inclusion != inclusion:
            f_max -= f
    f_list = np.linspace(0.01, f_max, 100)
    out_graph.clear_output()
    subplot_data = {}
    plotted = []

w_setgraph.on_click(compute_f_list)

# Ajout de modèles
display(widgets.Label(value="Select a model to plot and click the 'Add model' button."))
w_addmodel = widgets.Button(description="Add model")
display(widgets.HBox([select_model, w_addmodel]))

def plot_model(b):
    """
    Routine appelée lors de l'appui sur le bouton 'Add model'.
    Vérifie que le modèle choisi n'a pas déjà été ajouté au graphe et l'ajoute le cas échéant.
    Met à jour le graphe.
    """
    global subplot_data, plotted, f_list, inclusion, fig
    model = select_model.value
    if model not in plotted:
        f_old = microstructure.dict_inclusions[inclusion] # Sauvegarde de la valeur initiale
        plotted.append(model)
        # Calcul de la liste des comportements pour les différents f
        list_behaviors = {} # Format : {parameter: [values selon f]}
        for f in f_list:
            microstructure.change_fi(inclusion, f)
            h_behavior = model.compute_h_behavior(microstructure)
            for parameter, value in h_behavior.items():
                if parameter not in list_behaviors.keys():
                    # Le paramètre est rencontré pour la première fois
                    list_behaviors[parameter] = []
                list_behaviors[parameter].append(value)
        # Mise à jour de subplot_data
        for parameter, values in list_behaviors.items():
            data = [f_list, values, model.name]
            # Création de l'entrée correspondant au paramètre s'il s'agit du premier plot
            if parameter not in subplot_data.keys():
                subplot_data[parameter] = []
            subplot_data[parameter].append(data)
        # Annulation des modifications sur la microstructure
        microstructure.change_fi(inclusion, f_old)
        # Tracé des données
        draw_all_data(subplot_data)
                    
w_addmodel.on_click(plot_model)

# Ajout de données depuis un fichier texte
display(widgets.Label(value="Plot data from a text file. Input files are in the 'model_comparison' folder. See 'example.txt' for the format."))
list_files = listdir('model_comparison/')
w_file = widgets.Dropdown(options=[file for file in list_files if file.endswith('.txt')])
w_add_data = widgets.Button(description="Add data")
display(widgets.HBox([w_file, w_add_data]))

def plot_data(b):
    """
    Routine appelée lors de l'appui sur le bouton 'Add data'.
    Vérifie que le fichier choisi n'a pas déjà été ajouté au graphe et l'ajoute le cas échéant.
    Met à jour le graphe.
    """
    global subplot_data, plotted, fig
    file_name = w_file.value[:-4]
    if file_name not in plotted:
        plotted.append(file_name)
        df = pd.read_csv('model_comparison/'+w_file.value)
        # Récupération de la liste des fractions volumiques
        try:
            f_values = df['f']
        except KeyError:
            with out_graph:
                print("Wrong format")
        # Récupération des paramètres
        parameters = df.keys()
        for parameter in parameters:
            if parameter == 'f':
                continue
            # Création de l'entrée associée au paramètre
            if parameter not in subplot_data.keys():
                subplot_data[parameter] = []
            values = list(df[parameter])
            subplot_data[parameter].append([f_values, values, file_name])
        # Tracé du graphe
        draw_all_data(subplot_data)
    
w_add_data.on_click(plot_data)

# Bornes de Hashin
w_addbounds = widgets.Button(description="Add Hashin bounds")
display(widgets.Label(value="Add Hashin bounds to figures"))
display(w_addbounds)

def plot_bounds(b):
    """
    Routine appelée lors de l'appui sur le bouton 'Add Hashin bounds'.
    Vérifie que les bornes (str "bounds") n'ont pas été ajoutées aux graphes et les ajoute le cas échéant.
    Met à jour le graphe.
    """
    global subplot_data, plotted, f_list, inclusion
    if "bounds" not in plotted:
        plotted.append("bounds")
        f_old = microstructure.dict_inclusions[inclusion] # Sauvegarde de la valeur initiale
        # Calcul de la liste des comportements pour les différents f
        list_behaviors = {} # Format : {parameter: [values selon f]}
        for f in f_list:
            microstructure.change_fi(inclusion, f)
            h_bounds = microstructure.Hashin_bounds()
            for parameter, value in h_bounds.items():
                if parameter not in list_behaviors.keys():
                    # Le paramètre est rencontré pour la première fois
                    list_behaviors[parameter] = []
                list_behaviors[parameter].append(value)
        # Mise à jour de subplot_data
        for parameter, values in list_behaviors.items():
            parameter_key = parameter[:-3] # Permet de retirer 'inf' et 'sup'
            data = [f_list, values, "Hashin bound "+parameter]
            # Création de l'entrée correspondant au paramètre s'il s'agit du premier plot
            if parameter_key not in subplot_data.keys():
                subplot_data[parameter_key] = []
            subplot_data[parameter_key].append(data)
        # Annulation des modifications sur la microstructure
        microstructure.change_fi(inclusion, f_old)
        # Tracé des données
        draw_all_data(subplot_data)

w_addbounds.on_click(plot_bounds)

# Sauvegarde
n_fig = 0 # Identifiant de la figure
w_save = widgets.Button(description="Save figures")
w_filename = widgets.Text(value='fig0.pdf')
display(widgets.Label(value="Enter a valid file name with an extension (ex: .pdf, .png, .jpg) and click the save button to save the figures. The pdf file will be saved in the 'outputs' folder"))
display(widgets.HBox([w_filename, w_save]))

def save(b):
    """
    Routine appelée lors d'un appui sur le bouton 'Save figures', sauvegarde la figure dans le dossier 'outputs' avec le nom choisi par l'utilisateur.
    Incrémente le nom automatiquement.
    """
    global fig, n_fig
    filename = w_filename.value
    fig.savefig("outputs/"+filename)
    # Mise à jour automatique du nom
    n_fig += 1
    w_filename.value = 'fig{}.pdf'.format(n_fig)
    
w_save.on_click(save)

display(out_graph)

---

## IV- Calculs automatisés depuis un fichier texte
TODO : Description de la section

TODO : Décrire les fichiers compatibles et le format voulu, demander à l'utilisateur de mettre ses fichiers dans le dossier inputs 

TODO : Réfléchir à un format pertinent des fichiers d'entrée 

TODO : Ajout d'une barre de progression (utile pour les longs fichiers)

In [ ]:
list_inputs = [] # Liste des fichiers compatibles avec le format présents dans le dossier 'inputs'
folder = 'inputs/'

# Recherche de fichiers
def compatible_file(file_name, folder):
    """
    Fonction qui vérifies si un fichier donné 'file_name' (str) dans le dossier 'folder' correspond au format.
    Cette fonction vérifie uniquement si le fichier est bien un fichier texte et si sa première ligne est '*homogeneisation'.
    Renvoie True si le fichier est compatible et False sinon.
    """
    result = True # Initialisation
    # Test du nom du fichier
    if len(file_name)<5 or file_name[-4:]!='.txt':
        result = False
    else:
        # Lecture de la première ligne
        with open(folder+file_name, 'r') as file:
            line = file.readline()
            if line.strip() != '*homogeneisation':
                result = False
    return result

def check_files(folder = 'inputs/'):
    """
    Routine qui met à jour la liste 'list_inputs' des fichiers compatibles dans le dossier 'inputs'.
    Appelée lors de l'appui sur le bouton 'refresh list' plus bas.
    """
    global list_inputs
    list_inputs_raw = listdir(folder)
    list_inputs = [] # Réinitialisation de la liste
    for file_name in list_inputs_raw:
        if compatible_file(file_name, folder):
            list_inputs.append(file_name)

# Lecture des fichiers et calcul des comportement homogénéisés
def read_file(b):
    """
    Routine appelée lors d'un appui sur le bouton 'Generate output file'.
    Lis le nom du fichier choisi par l'utilisateur. Ouvre et lit le fichier.
    Affiche un message à l'utilisateur si une erreur est détectée.
    Sinon, calcule le comportement homogénéisé de chaque microstructure et crée un fichier de sortie dans le dossier 'outputs'.
    """
    folder_in = 'inputs/'
    folder_out = 'outputs/'
    file_name = w_file.value
    out_file.clear_output()
    read_matrix = False # Définit si la ligne lue correspond à la définition d'une nouvelle microstructure ou d'une inclusion.
    read_model = True # Définit si la ligne lue correspond à la définition d'un modèle.
    dict_inclusions = {}
    n = 0
    # Initialisation du fichier de sortie
    with open(folder_out+'out_'+file_name, 'w') as file_out:
        None
    # lecture du fichier d'entrée
    with open(folder_in+file_name, 'r') as file:
        lines = file.readlines()
        for n_line, line in enumerate(lines[1:]):
            try:
                if read_model:
                    # Définition du modèle
                    model_name = line.strip()
                    model = str_to_model(model_name)
                    # Passage à la ligne suivante
                    read_model = False
                    read_matrix = True
                elif read_matrix:
                    # Lecture du comportement de la matrice
                    matrix_behavior = {}
                    line1 = line.strip().split(',')
                    for parameter in line1:
                        parameter = parameter.split(':')
                        matrix_behavior[parameter[0]] = float(parameter[1])
                    # Passage à la ligne suivante
                    read_matrix = False
                elif line.strip() == '*':
                    # Calcul de la microstructure précédente
                    microstructure = Microstructure(matrix_behavior, dict_inclusions)
                    behavior_h = model.compute_h_behavior(microstructure)
                    # Écriture du comportement dans le fichier de sortie
                    with open(folder_out+'out_'+file_name, 'a') as file_out:
                        file_out.write('Essai {}\n'.format(n))
                        file_out.write(str(behavior_h)+'\n')
                    # Passage à la ligne suivante
                    read_model = True
                    dict_inclusions = {}
                    n += 1
                else:
                    # Lecture d'une inclusion
                    line1 = line.strip().split(',')
                    type_inclusion = line1[0]
                    f = line1[-1] # Fraction volumique
                    inclusion_behavior = {}
                    for parameter in line1[1:-1]:
                        parameter = parameter.strip().split(':')
                        inclusion_behavior[parameter[0]] = float(parameter[1])
                    # Génération de l'inclusion
                    inclusion = Inclusion(int(type_inclusion), inclusion_behavior)
                    dict_inclusions[inclusion] = float(f)
            except:
                with out_file:
                    print("Error on line {} : {} ".format(n_line+1, line))
                    return None
    with out_file:
        print("Output file generated in the 'outputs' folder ")

In [ ]:
def refresh(b):
    """
    Routine appelée lors d'un appui sur le bouton 'Refresh input files'. 
    Met à jour la liste des fichiers d'input compatibles et met à jour le widget de sélection.
    """
    check_files()
    w_file.options = list_inputs

b_refresh = widgets.Button(description='Refresh input files list')
display(b_refresh)
w_label = widgets.Label(value='Choose an input file :')
w_file = widgets.Dropdown(options=list_inputs)
refresh(None) # Mise à jour de la liste des fichiers d'entrée disponibles
b_compute = widgets.Button(description='Generate output file')
display(widgets.HBox([w_label, w_file, b_compute]))
out_file = widgets.Output(layout={'width': 'max-content', 'border': '1px solid #FF625BF5'})
display(out_file)
out_file.clear_output()
with out_file:
    print("Press 'Generate output file' to compute ")
    
b_refresh.on_click(refresh)
b_compute.on_click(read_file)

message = """If your file does not appear :
- Check that it is a '.txt' file and that its first line is '*homogeneisation',
- Check that your file is in the 'inputs' folder,
- Press the 'Refresh input files list' button."""

print(message)


---

## V- Description des modèles

Pour ajouter un modèle, écrire simplement sa description dans un fichier Markdown (.md) dans le dossier 'model_descriptions'. 

La première ligne doit être de la forme :

'# Nom du modèle'


In [ ]:
from IPython.display import Latex, Markdown
from os import listdir

# Récupération des fichiers de description des modèles  
folder = 'model_descriptions/'
folder_files = listdir(folder)
descriptions = [] # Liste des fichiers correspondants aux modèles décrits, sous la forme [('nom modèle', chemin_fichier)]
for file_name in folder_files:
    if file_name.endswith('.md'):
        path = folder + file_name
        with open(path, 'r') as opened_file:
            title = opened_file.readline()
        model_name = title[2:].strip() # Suppression des caractères '# ' au début du titre
        descriptions.append((model_name, path))

In [ ]:
# Affichage de la description
w_description = widgets.Dropdown(options=descriptions)
display(w_description)
out_description = widgets.Output(layout={'border': '1px solid #FF625BF5'})
display(out_description)

def display_description(change):
    """
    Fonction appelée lors d'un changement de value du widget w_description.
    Récupère le modèle choisi et affiche sa description.
    """
    out_description.clear_output()
    file_name = w_description.value
    with open(file_name, 'r') as file:
        description = file.read()
        with out_description:
            display(Markdown(description))
            
display_description(None)
w_description.observe(display_description, names='value')


---

# Tests 

## Initialisation micro

In [ ]:
### Paramètres de test ###
import time
K_target = 300
G_target = 10
Ki = 833.33
Gi = 384.62 
fi = 0.01
Km = 200
Gm = 1
inclusion = Inclusion(0, {'K': Ki, 'G': Gi})
microstructure = Microstructure({'K': Km, 'G': Gm}, {inclusion: fi})

## Premier algo

In [ ]:
# TODO : fonction qui change la fraction de matrice dans classes
t = time.time()

target = np.array([K_target, G_target])
# f_opti = 0.5 à peu près

def grad(target, model, microstructure, inclusion, error):
    """
    Renvoie le gradient en f de l'écart des moindres carrés entre le comportement cible et le comportement homogénéisé pour la microstructure et le modèle donnés.
    """
    pas = 0.001
    # Valeur de l'erreur en n+1
    microstructure.dict_inclusions[inclusion] += pas
    microstructure.f_matrix = microstructure.compute_fm()
    behavior = model.compute_h_behavior(microstructure)
    list_behavior = np.array([behavior['K'], behavior['G']])
    error1 = np.linalg.norm(list_behavior-target)
    # Retour à la microstructure de base
    microstructure.dict_inclusions[inclusion] -= pas
    microstructure.f_matrix = microstructure.compute_fm()
    return((error1-error)/pas)

def newton(target, model, microstructure, inclusion):
    """
    Réalise une descente du gradient pour trouver la fraction volumique idéale d'inclusion pour atteindre le comportement homogénéisé cible.
    """
    seuil = 10**(-5)
    max_iterations = 1000
    iteration = 0
    # Valeur de l'erreur
    behavior = model.compute_h_behavior(microstructure)
    list_behavior = np.array([behavior['K'], behavior['G']])
    error = np.linalg.norm(list_behavior-target)
    while error>seuil and iteration<max_iterations:
        grad_error = grad(target, model, microstructure, inclusion, error)
        iteration += 1
        # TODO : coder une optimisation de alpha
        f = microstructure.dict_inclusions[inclusion] - 0.01*np.sign(grad_error)
        if f>=1:
            f = 0.99
        elif f<=0:
            f = 0.01
        microstructure.dict_inclusions[inclusion] = f
        microstructure.f_matrix = microstructure.compute_fm()
        behavior = model.compute_h_behavior(microstructure)
        list_behavior = np.array([behavior['K'], behavior['G']])
        error = np.linalg.norm(list_behavior-target)
    print(target, behavior)
    return(microstructure)

inclusion = list(microstructure.dict_inclusions.keys())[0]
microstructure.dict_inclusions[inclusion] = 0.01
microstructure.f_matrix = microstructure.compute_fm()
print(microstructure)
model = Mori_Tanaka()
print(newton(target, model, microstructure, inclusion))

print("time ", time.time()-t)

## Initialisation micro

In [ ]:
### Paramètres de test ###
K_target = 300
G_target = 10
Ki = 833.33
Gi = 384.62 
fi = 0.01
Km = 200
Gm = 10
inclusion = Inclusion(0, {'K': Ki, 'G': Gi})
microstructure = Microstructure({'K': Km, 'G': Gm}, {inclusion: fi})

### Deuxième algo

In [ ]:
### Paramètres de test ###
t = time.time()
errors = []

import pickle as pk
                
### Paramètres de test ###
K_target = 300
G_target = 10
Ki = 833.33
Gi = 384.62 
fi = 0.01
Km = 200
Gm = 1
inclusion = Inclusion(0, {'K': Ki, 'G': Gi})
microstructure = Microstructure({'K': Km, 'G': Gm}, {inclusion: fi})

def fun(value_incr, maxi, mini):
    """
    Incrémente la valeur du paramètre. 
    Renvoie un booléen qui indique si la variable a été incrémentée ou non (c'est le cas lorsque la variable est en dehors des bornes min max)
    """
    inclusion = list(microstructure.dict_inclusions.keys())[0]
    value = microstructure.dict_inclusions[inclusion]
    value += value_incr
    changed = True # True si la variable a été modifiée, par défaut True
    if value > maxi:
        value = maxi
        changed = False
    if value < mini:
        value = mini
        changed = False
    microstructure.change_fi(inclusion, value)
    return changed

def fun_G(value_incr, maxi, mini):
    """
    Liée à G_matrice
    """
    value = microstructure.matrix_behavior['G']
    value += value_incr
    changed = True
    if value > maxi:
        value = maxi
        changed = False
    if value < mini:
        value = mini
        changed = False
    microstructure.matrix_behavior['G'] = value
    return changed
    
def fun_K(value_incr, maxi, mini):
    """
    Liée à K_matrice
    """
    value = microstructure.matrix_behavior['K']
    value += value_incr
    changed = True
    if value > maxi:
        value = maxi
        changed = False
    if value < mini:
        value = mini
        changed = False
    microstructure.matrix_behavior['K'] = value
    return changed
        
# ddl = {fun: [0.01, 0.99]}
# ddl = {fun_K: [0.01, 1000]}
ddl = {fun_G: [0.01, 1000], fun_K: [0.01, 1000]}
# ddl = {fun: [0.01, 0.99], fun_K: [0.01, 1000]}
model = Mori_Tanaka()
### ###

def compute_error(microstructure, model, target):
    """
    Calcule l'erreur au sens des moindres carrés entre le comportement cible et le comportement homogénéisé.
    """
    behavior_h = model.compute_h_behavior(microstructure)
    difference = [target[parameter]-behavior_h[parameter] for parameter in target.keys()]
    return np.linalg.norm(difference)

def grad(target, model, ddl, error_old):
    """
    target: dict, comportement cible
    model: instance de classe Model
    microstructure: instance de classe Microstructure
    ddl: dict des variables du problème d'optimisation avec leurs valeurs min et max
    error_old: float, erreur calculée pour le set de variables de l'itération en cours
    """
    global microstructure
    result = {} # format: {function associée à la variable: valeur_du_gradient}
    pas = 0.01
#     # Sauvegarde de la microstructure
#     with open('save', 'bw') as file:
#         pickler = pk.Pickler(file)
#         pickler.dump(microstructure)
    # Calcul du gradient terme à terme
    for function, min_max in ddl.items():
        mini, maxi = min_max # Bornes de la variable
        # TODO : Prendre en compte des inclusions mutliples
        pas_i = (maxi - mini)*pas # Pas spécifique au paramètre
#         function(pas_i, maxi, mini) # Mise à jour du paramètre
        changed = function(pas_i, maxi, mini) # Mise à jour du paramètre
        # Calcul de la nouvelle erreur
        error_new = compute_error(microstructure, model, target)
        # Calcul du gradient associé au paramètre
        result[function] = (error_new - error_old)/pas_i
        # Annulation des modifications sur la microstructure
#         with open('save', 'br') as file:
#             unpickler = pk.Unpickler(file)
#             microstructure = unpickler.load()
        if changed:
            function(-pas_i, maxi, mini)
    # Suppression du fichier de sauvegarde de la microstructure
#     !rm save
    return result

def newton(target, model, ddl):
    """
    Réalise une descente du gradient pour trouver la fraction volumique idéale d'inclusion pour atteindre le comportement homogénéisé cible.
    """
    global microstructure
    pas = 0.05
    seuil = 10**(-3)
    criteria = seuil + 1 # Initialisation du critère permettant de lancer la boucle d'optimisation
    max_iterations = 10000
    iteration = 0
    # Valeur de l'erreur
    error = compute_error(microstructure, model, target)
    while criteria>seuil and iteration<max_iterations:
        # Calcul du gradient
        grad_error = grad(target, model, ddl, error)
        #print("iter ", iteration, " error ", error," fi ", microstructure.dict_inclusions[inclusion]," grad ", list(grad_error.values())[0])
        iteration += 1
        # TODO : coder une optimisation de alpha
        # Mise à jour des variables
        for function, minmax in ddl.items():
            mini, maxi = minmax
            pas_i = pas#*(maxi-mini)
            value_incr = -pas_i*grad_error[function]
            function(value_incr, maxi, mini) # Calcul des variables à l'itération suivante
#             print("iter ", iteration, " error ", error," Gm ", microstructure.matrix_behavior['G']," grad ", list(grad_error.values())[0]," value_incr ", value_incr)
#             print("iter ", iteration, " error ", error," fi ", microstructure.dict_inclusions," grad ", list(grad_error.values())[0]," value_incr ", value_incr)
#             print("iter ", iteration, " error ", error," value_incr ", value_incr)
        # Mise à jour de l'erreur
        error = compute_error(microstructure, model, target)
        errors.append(error)
        if len(errors)>2:
            criteria = errors[-2] - errors[-1]
    behavior = model.compute_h_behavior(microstructure)
    print("iter ", iteration)
    print(target, behavior)
    return(microstructure)

# print(grad(target, model, microstructure, ddl, 80.7))
# print(compute_error(microstructure, model, target))

print(microstructure)
newton(target, model, ddl)
print(microstructure)

# print(microstructure)
# function = list(ddl.keys())[0]
# function(5,200,0)
# print(microstructure)

print("time ", time.time()-t)

plt.semilogy(errors)